In [1]:
import numpy as np
import tensorflow as tf
from collections import namedtuple 
##from libs.connections import  conv2d, linear 

def linear(x, n_units, scope=None, stddev=0.02,
           activation=lambda x: x):
    """Fully-connected network.
    Parameters
    ----------
    x : Tensor
        Input tensor to the network.
    n_units : int
        Number of units to connect to.
    scope : str, optional
        Variable scope to use.
    stddev : float, optional
        Initialization's standard deviation.
    activation : arguments, optional
        Function which applies a nonlinearity
    Returns
    -------
    x : Tensor
        Fully-connected output.
    """
    shape = x.get_shape().as_list()

    with tf.variable_scope(scope or "Linear"):
        matrix = tf.get_variable("Matrix", [shape[1], n_units], tf.float32,
                                 tf.random_normal_initializer(stddev=stddev))
        return activation(tf.matmul(x, matrix))
def conv2d(x, n_filters,
           k_h=5, k_w=5,
           stride_h=2, stride_w=2,
           stddev=0.02,
           activation=None,
           bias=True,
           padding='SAME',
           name="Conv2D"):
    """2D Convolution with options for kernel size, stride, and init deviation.
    Parameters
    ----------
    x : Tensor
        Input tensor to convolve.
    n_filters : int
        Number of filters to apply.
    k_h : int, optional
        Kernel height.
    k_w : int, optional
        Kernel width.
    stride_h : int, optional
        Stride in rows.
    stride_w : int, optional
        Stride in cols.
    stddev : float, optional
        Initialization's standard deviation.
    activation : arguments, optional
        Function which applies a nonlinearity
    padding : str, optional
        'SAME' or 'VALID'
    name : str, optional
        Variable scope to use.
    Returns
    -------
    x : Tensor
        Convolved input.
    """
    with tf.variable_scope(name):
        w = tf.get_variable(
            'w', [k_h, k_w, x.get_shape()[-1], n_filters],
            initializer=tf.truncated_normal_initializer(stddev=stddev))
        conv = tf.nn.conv2d(
            x, w, strides=[1, stride_h, stride_w, 1], padding=padding)
        if bias:
            b = tf.get_variable(
                'b', [n_filters],
                initializer=tf.truncated_normal_initializer(stddev=stddev))
            conv = tf.nn.bias_add(conv, b)
        if activation:
            conv = activation(conv)
        return conv

# 그래프 리셋
tf.reset_default_graph()
# 재현성을 위해 시드 지정
tf.set_random_seed(1)
# 자료 입력
from tensorflow.examples.tutorials.mnist import input_data
mnist=input_data.read_data_sets("MNIST_data/",one_hot=True)
learning_rate=0.0001; epochs=10; batch_size=100
X=tf.placeholder(tf.float32,[None,784]) 
X_img=tf.reshape(X,[-1,28,28,1]) 
Y=tf.placeholder(tf.float32,[None,10]) 
# ResNet 블록 구조(bottleneck 구조) 
LayerBlock = namedtuple('LayerBlock', ['num_repeats', 'num_filters', 'bottleneck_size'])
blocks = [LayerBlock(3, 128, 32),LayerBlock(3, 256, 64),LayerBlock(3, 512, 128),
          LayerBlock(3, 1024, 256)]
# 채널수 64의 합성곱 출력을 만들고 다운샘플링
net = conv2d(X_img, 64, k_h=7, k_w=7, name='conv1', activation=tf.nn.relu)
net = tf.nn.max_pool(net, [1, 3, 3, 1], strides=[1, 2, 2, 1], padding='SAME')
# ResNet 블록구조의 입력 생성 
net = conv2d(net, blocks[0].num_filters, k_h=1, k_w=1, stride_h=1, stride_w=1, 
      padding='VALID', name='conv2')
# ResNet 블록 반복 
for block_i, block in enumerate(blocks):
    for repeat_i in range(block.num_repeats):
        name = 'block_%d/repeat_%d' % (block_i, repeat_i)
        conv1 = conv2d(net, block.bottleneck_size, k_h=1, k_w=1, 
                         padding='VALID', stride_h=1, stride_w=1, 
                         activation=tf.nn.relu,name=name + '/conv_in')                      
        conv2 = conv2d(conv1, block.bottleneck_size, k_h=3, k_w=3,
                         padding='SAME', stride_h=1, stride_w=1,
                         activation=tf.nn.relu,name=name + '/conv_bottleneck')
        conv3 = conv2d(conv2, block.num_filters, k_h=1, k_w=1,
                         padding='VALID', stride_h=1, stride_w=1,
                         activation=tf.nn.relu, name=name + '/conv_out')

        net = conv3 + net
    try:
# upscale to the next block size
        next_block = blocks[block_i + 1]
        net = conv2d(net, next_block.num_filters, k_h=1, k_w=1,
                         padding='SAME', stride_h=1, stride_w=1, bias=False,
                         name='block_%d/conv_upscale' % block_i)
    except IndexError:
        pass
# 평균 풀링을 이용하여 블록 구조의 최종 출력의 차원 변환
net = tf.nn.avg_pool(net, ksize=[1, net.get_shape().as_list()[1],net.get_shape().as_list()[2], 1],
                     strides=[1, 1, 1, 1], padding='VALID')
#ResNet 블록 구조의 최종 출력을 1D로 변환
Flat=tf.reshape(net,[-1, net.get_shape().as_list()[1] *net.get_shape().as_list()[2] 
               *net.get_shape().as_list()[3]])
# 최종 출력을 위해 소프트맥스함수 지정
Y_pred =linear(Flat, 10, activation=tf.nn.softmax)
cost=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=Y_pred, labels=Y))
optim=tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
correct_predict=tf.equal(tf.argmax(Y_pred,1), tf.argmax(Y,1))
accuracy=tf.reduce_mean(tf.cast(correct_predict, tf.float32))
sess=tf.Session(); sess.run(tf.global_variables_initializer())
for epoch in range(epochs):
    avg_cost=0
    total_batch=int(mnist.train.num_examples/batch_size)
    for i in range(total_batch):
        batch_xs,batch_ys=mnist.train.next_batch(batch_size)
        feed_dict={X:batch_xs, Y:batch_ys}
        sess.run(optim, feed_dict=feed_dict)
        ccost=sess.run(cost, feed_dict=feed_dict)
        avg_cost+=ccost/total_batch
        acc=sess.run(accuracy, feed_dict=feed_dict)
    print('Epoch: %d' %(epoch+1),'cost= %f, accuracy= %f' %(avg_cost, acc))
# 훈련 데이터, 검정 데이터의 오분류율
acc_tr=0; acc_ts=0
for ii in range(100): #메모리 문제를 피하기 위해 자료를 100개로 분할
    xr,yr=mnist.train.next_batch(550)
    acc_tr= acc_tr+ 0.01*sess.run(accuracy, feed_dict={X:xr, Y:yr}) 
    xt,yt=mnist.test.next_batch(100)
    acc_ts= acc_ts+ 0.01*sess.run(accuracy, feed_dict={X:xt, Y:yt})    
print('misclassification error(tr):', 1-acc_tr)
print('misclassification error(ts):', 1-acc_ts)   

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Colocations handled automatically by placer.
Epoch: 1 cost= 2.061093, accuracy= 0.500000
Epoch: 2 cost= 1.837618, accuracy= 0.590000
Epoch: 3 cost= 1.773470, accuracy= 0.650000
Epoch: 4 cost= 1.764926, accuracy= 0.700000
Epoch: 5 cost= 1.695737, accuracy= 0.820000
Epoch: 6 